# Results Notebook

This notebook visualizes the results from the trained models. It contains the figures which are used in the report, which means that interpretations of the individual graphics are mostly done in the report itself and are not present in the notebook. First, the logged metrics during training are shown. Next, the validation results are presented. Further down, the metrics on the test sets are visualized.

In [ ]:
# import dependencies
import os
import git
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import json
import matplotlib.pyplot as plt


def get_git_root(path):
        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return git_root

os.chdir(get_git_root("."))

In [ ]:
# configuration of runs

ENTITY="simonluder"
PROJECT="MSE_P7"
ARTIFACT = "validation_results"

RUNS = [
    "2D_GeoShape_32_linear_tabular_1704819570",
    "2D_GeoShape_32_linear_cnn_image_1704819570",
    "2D_GeoShape_32_linear_clip_text_1704819570",
    "2D_GeoShape_32_linear_clip_image_1704819570",
    "2D_GeoShape_64_linear_tabular_1705056127",
    "2D_GeoShape_64_linear_cnn_image_1705051540",
    "2D_GeoShape_64_linear_clip_text_1705056262",
    "2D_GeoShape_64_linear_clip_image_1705056262", 
    "2D_GeoShape_sub100_32_linear_tabular_1705411529", 
    "2D_GeoShape_sub100_32_linear_cnn_image_1705405821", 
    "2D_GeoShape_sub100_32_linear_clip_text_1705410786",
    "2D_GeoShape_sub100_32_linear_clip_image_1705569629"
    
    ]

LOG_DIR = "runs/"

download = True

color_palette = {"clip_text": "#7852A9", "clip_image":"#80de81", "tabular": "#4285c6", "cnn_image":"#8FD4CB"}

## Load results

In [ ]:
# functions

def get_config(filepath):

    with open(filepath, 'r') as f:
        return json.load(f)
       

def get_metrics(filepath):

    with open(filepath, 'r') as f:
        data = json.load(f)

    data_train = []
    data_val = []
    data_test = []

    for entry in data:

        entry_train = entry.get("train")
        entry_val = entry.get("val")
        entry_test = entry.get("test")

        if entry_train:
            data_train.append(entry_train)

        if entry_val:
            data_val.append(entry_val)

        if entry_test:
            data_test.append(entry_test)

    return data_train, data_val, data_test

def postprocess_df_val(df_val):
    df = df_val.explode('samples')
    df = df.reset_index(drop=True)
    df_samples = pd.json_normalize(df["samples"])
    df = pd.concat([df.drop(columns=['samples']), df_samples], axis=1)

    df["path_original"] = df["path_original"].str.replace("/workspace", ".")
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df

def postprocess(df):
    
    pattern = r'(clip_text|clip_image|tabular|cnn_image)'
    df['encoder'] = df['run'].str.extract(pattern, expand=False)

    pattern = r'(_32_|_64_)'
    df['image_size'] = df['run'].str.extract(pattern, expand=False).str.replace("_", "").astype(int)

    pattern = r'(sub100)'
    df['subset'] = df['run'].str.extract(pattern, expand=False).str.replace("sub", "")
    df.loc[df['subset'].isna(), "subset"] = "1000"
    df['subset'] = df['subset'].astype(int)
    return df


In [ ]:
# load training datasets

ds_path = "./data"

# load dataset
datasets = list()
for dataset in os.listdir(ds_path):
    dataset_path = os.path.join(ds_path, dataset, "labels.csv")
    datasets.append(pd.read_csv(dataset_path))
df_datasets = pd.concat(datasets)


# load config
configs = list()
for run in RUNS:
    config = get_config(filepath = f"runs/{run}/config.json")
    configs.append(config)

df_config = pd.DataFrame.from_records(configs)
df_config["test_images"] = df_config["test_images"].str.replace("/workspace", ".")
df_config["test_labels"] = df_config["test_labels"].str.replace("/workspace", ".")


# load metrics
df_train_list = []
df_val_list = []
df_test_list = []
for run in RUNS:

    # load jsons
    data_train, data_val, data_test = get_metrics( filepath = f"runs/{run}/metrics.json" )

    df_train = pd.DataFrame.from_records(data_train)
    df_val = pd.DataFrame.from_records(data_val)
    df_test = pd.DataFrame.from_records(data_test)


    df_train["run"] = run
    df_val["run"] = run
    df_test["run"] = run

    # postprocessing
    if len(df_val):
        df_val = postprocess_df_val(df_val)

    # postprocessing
    if len(df_test):
        df_test = postprocess_df_val(df_test)

    df_train = postprocess(df_train)
    df_val = postprocess(df_val)
    df_test = postprocess(df_test)

    df_train_list.append(df_train)
    df_val_list.append(df_val)
    df_test_list.append(df_test)

df_train = pd.concat(df_train_list)
df_val = pd.concat(df_val_list)
df_test = pd.concat(df_test_list)


## Training

In [ ]:
df_train_32 = df_train.loc[(df_train["image_size"]==32) & (df_train["subset"]==1000)]

plt.figure(figsize=(10, 4))
sns.lineplot(data=df_train_32, x="epoch", y="epoch_loss", hue="encoder", palette=color_palette)
plt.title("MSE loss for the models trained on train_32")
plt.ylabel("MSE loss per epoch")
plt.yscale("log")

In [ ]:
df_train_64 = df_train.loc[(df_train["image_size"]==64) & (df_train["subset"]==1000)]

plt.figure(figsize=(10, 4))
sns.lineplot(data=df_train_64, x="epoch", y="epoch_loss", hue="encoder", palette=color_palette)
plt.title("MSE loss for the models train_64")
plt.ylabel("MSE loss per epoch")
plt.yscale("log")

In [ ]:
df_train_32_sub100 = df_train.loc[(df_train["image_size"]==32) & (df_train["subset"]==100)]

plt.figure(figsize=(10, 4))
sns.lineplot(data=df_train_32_sub100, x="epoch", y="epoch_loss", hue="encoder", palette=color_palette, alpha=0.6)
plt.title("MSE loss for the models trained on train_32_sub100")
plt.ylabel("MSE loss per epoch")
plt.yscale("log")

## Validation

The following tables shows the best epoch during training per model, measured by the intersection over union on the validation dataset. 

In [ ]:
# add shape information from the training datasets from train data to the validation samples
df_shapes_info = df_datasets[["im_res", "im_shape","randomize","shape_name","radius","x","y","rotation","aspect_ratio","fill_color","bg_color","file"]]
df_shapes_info_val = pd.merge(df_val, df_shapes_info, left_on='path_original', right_on='file')

# get best epoch per run
best_validation_epoch = df_shapes_info_val.iloc[df_shapes_info_val.groupby("run")["mean_iou"].idxmax()][["run", "epoch", "mean_iou"]]
best_validation_epoch
# print(best_validation_epoch.to_latex())

The following table shows all metrics on the validation dataset for the best epoch per model. 

In [ ]:
best_epoch_per_run = [(run, epoch) for run, epoch in zip(best_validation_epoch["run"].to_list(), best_validation_epoch["epoch"].to_list())]

df_best_valdation_results = df_shapes_info_val[df_shapes_info_val.apply(lambda row: (row['run'], row['epoch']) in best_epoch_per_run, axis=1)]
df_best_valdation_results = df_best_valdation_results.groupby(["run", "epoch"])[["IoU", "IoU_centered", "l2_distance", "abs_angle_diff", "abs_diameter_diff", "abs_contour_diff"]].mean()
df_best_valdation_results
# print(df_best_valdation_results.to_latex(float_format="%.3f"))

### Development of metrics during training

To better examine the individual encoder variants and understand the training of the models, the metrics are calculated every 25 epochs on the validation dataset. The following subsection shows a visualization per selected metric of all models in relation to the epoch in training. The metrics compare the generated geometric shapes with the ground truth.

In [ ]:
mean_val = df_shapes_info_val.groupby(["epoch", "run"])[["IoU", "IoU_centered", "l2_distance", "abs_angle_diff", "abs_diameter_diff", "abs_contour_diff"]].mean().reset_index()

mean_val = postprocess(mean_val)

mean_val_32 = mean_val.loc[(mean_val["image_size"]==32) & (mean_val["subset"]==1000)]


**Intersection over Union (IoU)**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="IoU", hue="encoder", palette=color_palette)
plt.title("Mean validation IoU during training")
plt.show()

**Figure Centered Intersection over Union (IoU)**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="IoU_centered", hue="encoder", palette=color_palette)
plt.title("Mean centered validation IoU during training")
plt.show()

**Centroid Distance**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="l2_distance", hue="encoder", palette=color_palette)
plt.title("Mean center distance during training")
plt.show()

**Mean absolute angular deviation of maximal diameter**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="abs_angle_diff", hue="encoder", palette=color_palette)
plt.title("Mean absolute angular deviation during training")
plt.show()

**Mean absolute difference in length of the maximum diameter**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="abs_diameter_diff", hue="encoder", palette=color_palette)
plt.title("Mean absolute diameter difference during training")
# plt.yscale("log")
plt.show()

**Mean absolute contour difference**

In [ ]:
plt.figure(figsize=(10, 3))
sns.lineplot(data=mean_val_32, x="epoch", y="abs_contour_diff", hue="encoder", palette=color_palette)
# plt.yscale("log")
plt.title("Mean contour difference during training")
plt.show()

## Testing

The following table shows the metrics obtained on the test data set. The model weights were selected for each model variant based on the best IoU on the validation dataset. 

In [ ]:
df_test_results = pd.merge(df_test, df_shapes_info, left_on='path_original', right_on='file')
test_results = df_test_results.groupby(["run", "epoch"])[["IoU", "IoU_centered", "l2_distance", "abs_angle_diff", "abs_diameter_diff", "abs_contour_diff"]].mean()
test_results
# print(test_results.to_latex(float_format="%.3f"))

In [ ]:
# df_test_results

# def create_metricx_boxplot(df, metric, title):
#     sns.boxplot(data=df, x="run", y=metric)
#     plt.title(title)
                


# create_metricx_boxplot(df_test_results, "l2_distance", "IoU")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
metrics = ["IoU", "IoU_centered", "l2_distance", "abs_angle_diff", "abs_diameter_diff", "abs_contour_diff"]
runs = df_test_results["run"].drop_duplicates()

runs = ["2D_GeoShape_32_linear_tabular_1704819570",
    "2D_GeoShape_32_linear_cnn_image_1704819570",
    "2D_GeoShape_32_linear_clip_text_1704819570",
    "2D_GeoShape_32_linear_clip_image_1704819570",]

runs = RUNS

df_cosine_similarities = list()

df = df_test_results
# df = df[["run", "path_original"] + metrics]
for i, run1 in enumerate(runs):
    for j, run2 in enumerate(runs):

        if i == j:
            continue

        print(run1, run2)

        df1 = df.loc[df["run"]==run1]
        df2 = df.loc[df["run"]==run2]

        for i, row1 in df1.iterrows():
  
            sample = row1["path_original"].split("/")[-1]

            v1 = df1.loc[df1["path_original"].str.contains(sample)][metrics].reset_index(drop=True)
            v2 = df2.loc[df2["path_original"].str.contains(sample)][metrics].reset_index(drop=True)

            # print(np.sum((v1.isna().values)))

            
            if np.sum((v1.isna().values)) + np.sum((v2.isna().values)) == 0:
                similarity = cosine_similarity(v1, v2)[0][0]

                df_cosine_similarities.append({"run1":run1, "run2":run2, "sample":sample, "score":similarity})
            
df_cosine_similarities = pd.DataFrame(df_cosine_similarities)


In [ ]:
cosine_similarity_mean = df_cosine_similarities.groupby(["run1", "run2"])["score"].mean().reset_index()
cosine_similarity_std = df_cosine_similarities.groupby(["run1", "run2"])["score"].std().reset_index()

cosine_mean_matrix = pd.DataFrame()
cosine_similarity_mean

for i, row in cosine_similarity_mean.iterrows():
    cosine_mean_matrix.loc[row["run1"], row["run2"]] = row["score"]

sns.heatmap(cosine_mean_matrix)

cosine_mean_matrix

### IoU on different shape types

Intersection over Union as boxplot per shape. The IoU is combined for all model variants.

In [ ]:
df_test_results
# rest_results_per_shape = df_test_results.groupby(["run", "shape_name"])[["IoU", "IoU_centered"]].mean().reset_index()
rest_results_per_shape = df_test_results
rest_results_per_shape = postprocess(rest_results_per_shape)


pattern = r'(sub100_32_linear|32_linear|64_linear)'
rest_results_per_shape['variant'] = rest_results_per_shape['run'].str.extract(pattern, expand=False)
rest_results_per_shape['testset, trainset'] = rest_results_per_shape['variant'].replace({"sub100_32_linear":"test32, train32_sub100",
                                                                                             "32_linear":"test32, train32",
                                                                                             "64_linear":"test64, train64"})
df_test_results['encoder, testset, trainset'] = df_test_results["encoder"] + ", " + df_test_results['testset, trainset']

# Create a boxplot
def boxplot_iou_per_shape(df, y):
    order = ['circle', 'triangle', 'square', 'pentagon', 'hexagon', 'heptagon', 'octagon', 'nonagon', 'star']
    plt.figure(figsize=(12,4))
    ax = sns.boxplot(x='shape_name', y=y, data=df, order=order)
    plt.title(f'Mean IoU per shape on the testing set')
    plt.xlabel("shape type")
    plt.ylabel(y)
    plt.show()


boxplot_iou_per_shape(rest_results_per_shape, y="IoU")

In [ ]:

# Create a boxplot
def boxplot_iou_per_shape(df, y):
    order = ['circle', 'triangle', 'square', 'pentagon', 'hexagon', 'heptagon', 'octagon', 'nonagon', 'star']
    plt.figure(figsize=(12,4))
    hue_order = ["clip_image, test32, train32",
                 "clip_image, test32, train32_sub100",
                 "clip_image, test64, train64",
                 "clip_text, test32, train32",
                 "clip_text, test32, train32_sub100",
                 "clip_text, test64, train64",
                 "cnn_image, test32, train32",
                 "cnn_image, test32, train32_sub100",
                 "cnn_image, test64, train64",
                 "tabular, test32, train32",
                 "tabular, test32, train32_sub100",
                 "tabular, test64, train64",
                 ]
    ax = sns.boxplot(x='shape_name', y=y, data=df, order=order, hue='encoder, testset, trainset', hue_order=hue_order)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.title(f'Mean IoU per shape on the testing set')
    plt.xlabel("shape type")
    plt.ylabel(y)
    plt.show()


boxplot_iou_per_shape(rest_results_per_shape, y="IoU")

### Centroid adjusted IoU on different shape types

In [ ]:
import math

def show_bin_prop_diameter(df, step=10):

    plt.figure(figsize=(12,5))

    vmin = math.floor(df['prop_diameter'].min() / step) 
    vmax = math.ceil(df['prop_diameter'].max() / step) 
    bins = np.arange(vmin, vmax + 1, 1) * step

    df['prop_diameter_binned'] = pd.cut(df['prop_diameter'], bins).sort_values()
  
    df = df.groupby(['encoder, testset, trainset', "prop_diameter_binned"])["IoU"].mean().reset_index()
    df['prop_diameter_binned'] = df['prop_diameter_binned'].astype(str)

    ax = sns.lineplot(data=df, x="prop_diameter_binned", y="IoU", hue='encoder, testset, trainset')
    sns.scatterplot(data=df, x="prop_diameter_binned", y="IoU", hue='encoder, testset, trainset', legend=False)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.xticks(rotation = -90)

    plt.title("Intersection over Union in relation to shape diameter")
    plt.ylabel("IoU")
    plt.xlabel("propotional diameter")
    plt.show()

df_test_results["prop_diameter"] = (df_test_results["radius"] * 2) / 256
df_test_results = postprocess(df_test_results)
df_test_results["variant"] = df_test_results['encoder'].astype(str) + ", size:" + df_test_results['image_size'].astype(str) + ", samples:" + df_test_results['subset'].astype(str)


pattern = r'(sub100_32_linear|32_linear|64_linear)'
df_test_results['variant'] = df_test_results['run'].str.extract(pattern, expand=False)
df_test_results['datasets: test, train'] = df_test_results['variant'].replace({"sub100_32_linear":"test32, train32_sub100",
                                                                                             "32_linear":"test32, train32",
                                                                                             "64_linear":"test64, train64"})
df_test_results['encoder, testset, trainset'] = df_test_results["encoder"] + ", " + df_test_results['datasets: test, train']

show_bin_prop_diameter(df_test_results, step=0.05)

In [ ]:
def show_bin_prop_diameter(df, step=10):

    plt.figure(figsize=(12,5))

    vmin = math.floor(df['prop_diameter'].min() / step) 
    vmax = math.ceil(df['prop_diameter'].max() / step) 
    bins = np.arange(vmin, vmax + 1, 1) * step

    df['prop_diameter_binned'] = pd.cut(df['prop_diameter'], bins).sort_values()
  
    df = df.groupby(['encoder, testset, trainset', "prop_diameter_binned"])["IoU_centered"].mean().reset_index()
    df['prop_diameter_binned'] = df['prop_diameter_binned'].astype(str)

    ax = sns.lineplot(data=df, x="prop_diameter_binned", y="IoU_centered", hue='encoder, testset, trainset')
    sns.scatterplot(data=df, x="prop_diameter_binned", y="IoU_centered", hue='encoder, testset, trainset', legend=False)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.xticks(rotation = -90)

    plt.title("Center adjusted Intersection over Union in relation to shape diameter")
    plt.ylabel("IoU_centered")
    plt.xlabel("propotional diameter")
    plt.show()


show_bin_prop_diameter(df_test_results, step=0.05)

In [ ]:
import math

def test(df, step=10):

    plt.figure(figsize=(12,5))
    


    vmin = math.floor(df['prop_diameter'].min() / step) 
    vmax = math.ceil(df['prop_diameter'].max() / step) 
    bins = np.arange(vmin, vmax + 1, 1) * step

    df['prop_diameter_binned'] = pd.cut(df['prop_diameter'], bins).sort_values()
        
    df = df.groupby(["run", "prop_diameter_binned", "image_size"])["IoU"].mean().reset_index()
    df['prop_diameter_binned'] = df['prop_diameter_binned'].astype(str)

    sns.boxplot(x='prop_diameter_binned', y="IoU", data=df, hue="image_size")
    plt.xticks(rotation = -90)

    plt.title("Intersection over Union in relation to shape diameter")
    plt.xlabel("proportional diameter")


iou_per_size = df_test_results.loc[~df_test_results["run"].str.contains("sub100")]


test(iou_per_size, step=0.05)

## Qualitative Evaluation on the Test Dataset



In [ ]:
import textwrap
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec

test_samples = df_test_results["path_original"].apply(lambda x: x.split("/")[-1]).drop_duplicates().to_list()[0:9]

fig, ax = plt.subplots(len(test_samples), len(RUNS) + 1, figsize=(13, 1 * len(test_samples)))
outergs = gridspec.GridSpec(1, 1)



for i, sample in enumerate(test_samples):
    for j, run in enumerate(RUNS):
        sub_df = df_test_results.loc[df_test_results["path_original"].str.contains(sample) &  df_test_results["run"].str.contains(run)]
        image_file = sub_df["path_generated"].values[0]
        variant = sub_df["encoder, testset, trainset"].values[0]      

        image_file = sub_df["path_generated"].values[0]
        im = cv2.imread(image_file)
        ax[i, j+1].imshow(im)

        if i == 0:
            ax[i, j+1].set_title(variant.replace(" test", "\ntest"), rotation=-90, size=10)

        if j == 0:
            image_file_gt = os.path.join("data/test256/images/", sample)
            im = cv2.imread(image_file_gt)
            ax[i, j].imshow(im)

            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])

            txt = ax[i, j].text(-30, 120, textwrap.fill(sample, 20), horizontalalignment='right', verticalalignment='center', wrap=True)

            if i == 0:
                ax[i, j].set_title("ground truth\n@ 256 x 256", rotation=-90, size=10)

                

        ax[i, j+1].set_xticks([])
        ax[i, j+1].set_yticks([])

rect = Rectangle((0.18, 1.03), -0.059, -0.93, facecolor='yellow', edgecolor='none',
                 transform=fig.transFigure, zorder=-1)
fig.patches.append(rect)


plt.show()

In [ ]:
import textwrap
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec

test_samples = df_test_results["path_original"].apply(lambda x: x.split("/")[-1]).drop_duplicates().to_list()

fig, ax = plt.subplots(len(test_samples), len(RUNS) + 1, figsize=(13, 1 * len(test_samples)))
outergs = gridspec.GridSpec(1, 1)



for i, sample in enumerate(test_samples):
    for j, run in enumerate(RUNS):
        sub_df = df_test_results.loc[df_test_results["path_original"].str.contains(sample) &  df_test_results["run"].str.contains(run)]
        image_file = sub_df["path_generated"].values[0]
        variant = sub_df["encoder, testset, trainset"].values[0]      

        image_file = sub_df["path_generated"].values[0]
        im = cv2.imread(image_file)
        ax[i, j+1].imshow(im)

        if i == 0:
            ax[i, j+1].set_title(variant.replace(" test", "\ntest"), rotation=-90, size=10)

        if j == 0:
            image_file_gt = os.path.join("data/test256/images/", sample)
            im = cv2.imread(image_file_gt)
            ax[i, j].imshow(im)

            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])

            txt = ax[i, j].text(-30, 120, textwrap.fill(sample, 20), horizontalalignment='right', verticalalignment='center', wrap=True)

            if i == 0:
                ax[i, j].set_title("ground truth\n@ 256 x 256", rotation=-90, size=10)

                

        ax[i, j+1].set_xticks([])
        ax[i, j+1].set_yticks([])

rect = Rectangle((0.18, 1.03), -0.059, -0.93, facecolor='yellow', edgecolor='none',
                 transform=fig.transFigure, zorder=-1)
fig.patches.append(rect)


plt.show()

In [ ]:
def split_list(lst, n):
    return [lst[i::n] for i in range(0, n, 1)]


test_samples = df_test_results["path_original"].apply(lambda x: x.split("/")[-1]).drop_duplicates().to_list()
test_samples = split_list(test_samples, 9)



for test_samples_figure in test_samples:
    fig, ax = plt.subplots(len(test_samples[0]), len(RUNS) + 1, figsize=(13, 1 * len(test_samples[0])))
    outergs = gridspec.GridSpec(1, 1)
    for i, sample in enumerate(test_samples_figure):
        for j, run in enumerate(RUNS):
            sub_df = df_test_results.loc[df_test_results["path_original"].str.contains(sample) &  df_test_results["run"].str.contains(run)]
            image_file = sub_df["path_generated"].values[0]
            variant = sub_df["encoder, testset, trainset"].values[0]      

            image_file = sub_df["path_generated"].values[0]
            im = cv2.imread(image_file)
            ax[i, j+1].imshow(im)

            if i == 0:
                ax[i, j+1].set_title(variant.replace(" test", "\ntest"), rotation=-90, size=10)

            if j == 0:
                image_file_gt = os.path.join("data/test256/images/", sample)
                im = cv2.imread(image_file_gt)
                ax[i, j].imshow(im)

                ax[i, j].set_xticks([])
                ax[i, j].set_yticks([])

                txt = ax[i, j].text(-30, 120, textwrap.fill(sample, 20), horizontalalignment='right', verticalalignment='center', wrap=True)

                if i == 0:
                    ax[i, j].set_title("ground truth\n@ 256 x 256", rotation=-90, size=10)

                    

            ax[i, j+1].set_xticks([])
            ax[i, j+1].set_yticks([])

    rect = Rectangle((0.18, 1.03), -0.059, -0.93, facecolor='yellow', edgecolor='none',
                    transform=fig.transFigure, zorder=-1)
    fig.patches.append(rect)


    plt.show()